In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c LANL-Earthquake-Prediction

Unzip the dataset

In [ ]:
!unzip /content/LANL-Earthquake-Prediction.zip

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import skew
from scipy.stats import kurtosis
from numpy import cov
from scipy.stats import pearsonr
from scipy.stats import kurtosis , skew
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import mean_absolute_error

In [ ]:
#Top 10 important features 
lst = ['2_Std300',
 '3_skew80',
 '2_min80',
 '3_skew300',
 '1_skew10',
 '2_Std80',
 '3_Mean10',
 '2_skew10',
 '2_min300',
 '2_90th precentile10']

In [ ]:
def fn1(df):
  #outliers
  dataframe = df[(df["acoustic_data"] >-2.0) & (df["acoustic_data"]<41.0)]

  def feat_transform(x):
    features = {}
    features['Mean'] = x.mean()
    features['Std'] = x.std()
    features['kurtosis']  = x.kurtosis()
    features['skew'] = x.skew()
    features['min'] = x.min()
    features['max'] = x.max()
    features['median'] = x.median()
    features["10th precentile"] = np.percentile(x, 10)
    features["50th precentile"] = np.percentile(x, 50)
    features["90th precentile"] = np.percentile(x, 90)

    for j in [10 , 80 , 300]:
      me = x.rolling(j).mean().dropna()
      features["1_Mean" + str(j)] = me.mean()
      features["1_Std" + str(j)] = me.std()
      features["1_kurtosis" + str(j)] = kurtosis(me)
      features["1_skew" + str(j)] = skew(me)
      features["1_min" + str(j)] = me.min()
      features["1_max" + str(j)] = me.max()
      features["1_median" + str(j)] = me.median()
      features["1_10th precentile"+ str(j)] = np.percentile(me, 10)
      features["1_50th precentile"+ str(j)] = np.percentile(me, 50)
      features["1_90th precentile"+ str(j)] = np.percentile(me, 90)

      st = x.rolling(j).std().dropna()
      features["2_Mean" + str(j)] = st.mean()
      features["2_Std" + str(j)] = st.std()
      features["2_kurtosis" + str(j)] = kurtosis(st)
      features["2_skew" + str(j)] = skew(st)
      features["2_min" + str(j)] = st.min()
      features["2_max" + str(j)] = st.max()
      features["2_median" + str(j)] = st.median()
      features["2_10th precentile"+ str(j)] = np.percentile(st, 10)
      features["2_50th precentile"+ str(j)] = np.percentile(st, 50)
      features["2_90th precentile"+ str(j)] = np.percentile(st, 90)


      skewness = x.rolling(j).skew().dropna()
      features["3_Mean" + str(j)] = skewness.mean()
      features["3_Std" + str(j)] = skewness.std()
      features["3_kurtosis" + str(j)] = kurtosis(skewness)
      features["3_skew" + str(j)] = skew(skewness)
      features["3_min" + str(j)] = skewness.min()
      features["3_max" + str(j)] = skewness.max()
      features["3_median" + str(j)] = skewness.median()
      features["3_10th precentile"+ str(j)] = np.percentile(skewness, 10)
      features["3_50th precentile"+ str(j)] = np.percentile(skewness, 50)
      features["3_90th precentile"+ str(j)] = np.percentile(skewness, 90)
    
    return features 
  print("from tqdm import tqdm")
  from tqdm import tqdm
  feature = []
  y = []
  for i in tqdm(range(int(dataframe.shape[0]/15000))):
    x = dataframe.iloc[i*15000 :i*15000 + 15000]
    y.append(x["time_to_failure"].values[-1])
    
    feature.append(feat_transform(x["acoustic_data"]))
    feat = pd.DataFrame(feature)

    feat["time_to_failure"] = y

  new_features = feat.loc[: , lst]
  
  x = new_features.iloc[: , :]

  scalar = joblib.load('scalar.pkl')
  x = scalar.transform(x)
  
   
  knn_model = joblib.load('Knn.pkl')
  RF_model = joblib.load('Forest.pkl')
  XGB_model = joblib.load('RBG.pkl')

  pred = (knn_model.predict(x) + RF_model.predict(x) + XGB_model.predict(x)) / 3
  return pred , y


In [ ]:
df = pd.read_csv("train.csv",nrows=10000000)

In [ ]:
pred , y = fn1(df)

from tqdm import tqdm


100%|██████████| 630/630 [00:23<00:00, 27.18it/s]


In [ ]:
def fn2(df):
  pred , y = fn1(df)
  return mean_absolute_error(y, pred)


In [ ]:
fn2(df)

from tqdm import tqdm


100%|██████████| 630/630 [00:23<00:00, 26.65it/s]


2.5446473884224146

#On test data

In [ ]:
def fn1(df):
  #outliers
  dataframe = df[(df["acoustic_data"] >-2.0) & (df["acoustic_data"]<41.0)]

  def feat_transform(x):
    features = {}
    features['Mean'] = x.mean()
    features['Std'] = x.std()
    features['kurtosis']  = x.kurtosis()
    features['skew'] = x.skew()
    features['min'] = x.min()
    features['max'] = x.max()
    features['median'] = x.median()
    features["10th precentile"] = np.percentile(x, 10)
    features["50th precentile"] = np.percentile(x, 50)
    features["90th precentile"] = np.percentile(x, 90)

    for j in [10 , 80 , 300]:
      me = x.rolling(j).mean().dropna()
      features["1_Mean" + str(j)] = me.mean()
      features["1_Std" + str(j)] = me.std()
      features["1_kurtosis" + str(j)] = kurtosis(me)
      features["1_skew" + str(j)] = skew(me)
      features["1_min" + str(j)] = me.min()
      features["1_max" + str(j)] = me.max()
      features["1_median" + str(j)] = me.median()
      features["1_10th precentile"+ str(j)] = np.percentile(me, 10)
      features["1_50th precentile"+ str(j)] = np.percentile(me, 50)
      features["1_90th precentile"+ str(j)] = np.percentile(me, 90)

      st = x.rolling(j).std().dropna()
      features["2_Mean" + str(j)] = st.mean()
      features["2_Std" + str(j)] = st.std()
      features["2_kurtosis" + str(j)] = kurtosis(st)
      features["2_skew" + str(j)] = skew(st)
      features["2_min" + str(j)] = st.min()
      features["2_max" + str(j)] = st.max()
      features["2_median" + str(j)] = st.median()
      features["2_10th precentile"+ str(j)] = np.percentile(st, 10)
      features["2_50th precentile"+ str(j)] = np.percentile(st, 50)
      features["2_90th precentile"+ str(j)] = np.percentile(st, 90)


      skewness = x.rolling(j).skew().dropna()
      features["3_Mean" + str(j)] = skewness.mean()
      features["3_Std" + str(j)] = skewness.std()
      features["3_kurtosis" + str(j)] = kurtosis(skewness)
      features["3_skew" + str(j)] = skew(skewness)
      features["3_min" + str(j)] = skewness.min()
      features["3_max" + str(j)] = skewness.max()
      features["3_median" + str(j)] = skewness.median()
      features["3_10th precentile"+ str(j)] = np.percentile(skewness, 10)
      features["3_50th precentile"+ str(j)] = np.percentile(skewness, 50)
      features["3_90th precentile"+ str(j)] = np.percentile(skewness, 90)
    
    return features 

  feature = feat_transform(dataframe)
  feat = pd.DataFrame(feature)
  new_features = feat.loc[: , lst]
  
  x = new_features.iloc[: , :]

  scalar = joblib.load('scalar.pkl')
  x = scalar.transform(x)
  
   
  knn_model = joblib.load('Knn.pkl')
  RF_model = joblib.load('Forest.pkl')
  XGB_model = joblib.load('RBG.pkl')

  pred = (knn_model.predict(x) + RF_model.predict(x) + XGB_model.predict(x)) / 3
  return float(pred)


In [ ]:
os.listdir('/content/test')

In [ ]:
f = os.listdir('/content/test')

In [ ]:
len(f)

2624

In [ ]:
for i in f:
  print(i.split(".")[0])
  break

seg_37608b


In [ ]:
x = pd.read_csv("/content/some_rows.csv")
p = fn1(x)


In [ ]:
p

6.2157251746065585

In [ ]:
file = []
pred = []
for i in f:
    x = pd.read_csv('/content/test/' + i)
    p = fn1(x)
    pred.append(p)
    file.append(i.split('.')[0])


In [ ]:
final_df = pd.DataFrame(list(zip(file, pred)),
               columns =['seg_id', 'time_to_failure'])
final_df.head(3)

,seg_id,time_to_failure
0,seg_37608b,7.333508
1,seg_827804,4.362303
2,seg_c20c06,4.679721


In [ ]:
final_df.to_csv("Lanl_pred.csv",index=False)